In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import datetime

In [2]:
data = pd.read_csv('movie_bd_v5.csv', encoding='utf-8')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1463,tt0479143,24000000,155721132,Rocky Balboa,Sylvester Stallone|Burt Young|Milo Ventimiglia...,Sylvester Stallone,It ain't over 'til it's over.,When he loses a highly publicized virtual boxi...,102,Drama,Columbia Pictures|Revolution Studios|Rogue Mar...,12/20/2006,6.4,2006
997,tt1659337,13000000,33400000,The Perks of Being a Wallflower,Logan Lerman|Emma Watson|Ezra Miller|Paul Rudd...,Stephen Chbosky,We are infinite.,A coming-of-age story based on the best-sellin...,102,Drama|Romance,Summit Entertainment|Mr. Mudd Production,9/20/2012,7.6,2012
604,tt1099212,37000000,392616625,Twilight,Kristen Stewart|Robert Pattinson|Billy Burke|T...,Catherine Hardwicke,"When you can live forever, what do you live for?",When Bella Swan moves to a small town in the P...,122,Adventure|Fantasy|Drama|Romance,Summit Entertainment|Maverick Films|Imprint En...,11/20/2008,5.8,2008
386,tt1250777,28000000,96188903,Kick-Ass,Aaron Taylor-Johnson|ChloÃ« Grace Moretz|Chris...,Matthew Vaughn,Shut up. Kick ass.,Dave Lizewski is an unnoticed high school stud...,117,Action|Crime,Lions Gate,3/22/2010,7.0,2010
2,tt2908446,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,Robert Schwentke,One Choice Can Destroy You,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,3/18/2015,6.3,2015


# Предобработка

In [38]:
answers = {}

# Backing up read data
original_data = data.copy()
df = data.copy()

# Converting types
df.release_date = df.release_date.apply(lambda release_date: datetime.datetime.strptime(release_date, '%m/%d/%Y'))
df.cast = df.cast.apply(lambda act: act.split(sep = '|'))
df.genres = df.genres.apply(lambda genre: genre.split(sep = '|'))
df.production_companies = df.production_companies.apply(lambda pc: pc.split(sep = '|'))
df.director = df.director.apply(lambda director: director.split(sep = '|'))

# Adding calculated columns
df['profit'] = df.revenue - df.budget
df['release_month'] = df.release_date.map(lambda release_date: release_date.month)

# There is a contradiction - the task needs order number in answers and forbids to use the list of answers in the code
def format_movie(answer_as_series):
    return '{}. {} ({})'.format('Вы думаете и в жизни у вас будут варианты ответов?', answer_as_series.original_title, answer_as_series.imdb_id)

# Accepts data frame and a function to get array value from a series.
def flat_map(df, f):
    return [x for row in df.itertuples() for x in f(row)]

# Sorts dictionary keys by according values
def sort_dictionary_by_value(dictionary):
    return {k: v for k, v in sorted(dictionary.items(), reverse=True, key=lambda item: item[1])}

def count_by_collection_item(df, collection_provider, count_provider=lambda row: 1):
    count_by_collection_item = Counter()
    for row in df.itertuples():
        for director in collection_provider(row):
            count_by_collection_item[director] += count_provider(row)
    return count_by_collection_item


# Checking data quality
df.info()
display(df.sample(5))
# df.imdb_id.nunique()        # 1889
# df.original_title.nunique() # 1887


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   imdb_id               1889 non-null   object        
 1   budget                1889 non-null   int64         
 2   revenue               1889 non-null   int64         
 3   original_title        1889 non-null   object        
 4   cast                  1889 non-null   object        
 5   director              1889 non-null   object        
 6   tagline               1889 non-null   object        
 7   overview              1889 non-null   object        
 8   runtime               1889 non-null   int64         
 9   genres                1889 non-null   object        
 10  production_companies  1889 non-null   object        
 11  release_date          1889 non-null   datetime64[ns]
 12  vote_average          1889 non-null   float64       
 13  release_year      

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,release_month
348,tt1225822,8000000,10848783,Extract,"[Jason Bateman, Mila Kunis, Kristen Wiig, Ben ...",[Mike Judge],Sticking it to the man has never looked so good.,The owner of a factory that produces flavor ex...,92,[Comedy],[Miramax Films],2009-09-03,5.8,2009,2848783,9
166,tt2883512,11000000,45967935,Chef,"[Jon Favreau, John Leguizamo, Bobby Cannavale,...",[Jon Favreau],Starting from scratch never tasted so good,A chef who loses his restaurant job starts up ...,115,[Comedy],"[Open Road Films, Aldamisa Entertainment, Kilb...",2014-05-08,7.2,2014,34967935,5
1136,tt0318155,80000000,68514844,Looney Tunes: Back in Action,"[Brendan Fraser, Jenna Elfman, Steve Martin, T...",[Joe Dante],Real life has never been so animated.,Bugs Bunny and Daffy Duck are up to their feud...,90,"[Animation, Comedy, Family]","[Lonely Film Productions GmbH & Co. KG., Warne...",2003-11-14,5.5,2003,-11485156,11
1212,tt1091191,40000000,149295601,Lone Survivor,"[Mark Wahlberg, Taylor Kitsch, Emile Hirsch, B...",[Peter Berg],Based on True Acts of Courage,"Based on the failed June 28, 2005 mission ""Ope...",121,"[Action, Drama, Thriller, War]","[Universal Pictures, Weed Road Pictures, Herri...",2013-12-24,7.4,2013,109295601,12
1737,tt0425413,10000000,33000000,Run Fatboy Run,"[Simon Pegg, Thandie Newton, Hank Azaria, Dyla...",[David Schwimmer],Love. Commitment. Responsibility. There's noth...,Five years after jilting his pregnant fiancÃ©e...,100,"[Comedy, Romance]","[Entertainment Films, Material Entertainment, ...",2007-09-06,6.2,2007,23000000,9


# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [4]:
found = df[df.budget == df.budget.max()].iloc[0]
answers['1'] = format_movie(found)
print(answers['1'])

Вы думаете и в жизни у вас будут варианты ответов?. Pirates of the Caribbean: On Stranger Tides (tt1298650)


ВАРИАНТ 2

In [5]:
found = df.query('budget == ' + str(df.budget.max())).iloc[0]

answers['1'] = format_movie(found)
print(answers['1'])

Вы думаете и в жизни у вас будут варианты ответов?. Pirates of the Caribbean: On Stranger Tides (tt1298650)


# 2. Какой из фильмов самый длительный (в минутах)?

In [6]:
found = df[df.runtime == df.runtime.max()].iloc[0]

answers['2'] = format_movie(found)
print(answers['2'], found.runtime)

Вы думаете и в жизни у вас будут варианты ответов?. Gods and Generals (tt0279111) 214


# 3. Какой из фильмов самый короткий (в минутах)?





In [7]:
found = df[df.runtime == df.runtime.min()].iloc[0]

answers['3'] = format_movie(found)
print(answers['3'], found.runtime)

Вы думаете и в жизни у вас будут варианты ответов?. Winnie the Pooh (tt1449283) 63


# 4. Какова средняя длительность фильмов?


In [8]:
answers['4'] = int(round(df.describe().loc['mean'].runtime))
print(answers['4'])

110


# 5. Каково медианное значение длительности фильмов? 

In [9]:
answers['5'] = int(round(df.describe().loc['50%'].runtime))
print(answers['5'])

107


# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [10]:
found = df[df.profit == df.profit.max()].iloc[0]

answers['6'] = format_movie(found)
print(answers['6'], found.profit, found.budget)


Вы думаете и в жизни у вас будут варианты ответов?. Avatar (tt0499549) 2544505847 237000000


# 7. Какой фильм самый убыточный? 

In [11]:
found = df[df.profit == df.profit.min()].iloc[0]

answers['7'] = format_movie(found)
print(answers['7'], found.profit, found.budget)


Вы думаете и в жизни у вас будут варианты ответов?. The Lone Ranger (tt1210819) -165710090 255000000


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [12]:
answers['8'] = len(df[df.profit > 0])
print(answers['8'])

1478


# 9. Какой фильм оказался самым кассовым в 2008 году?

In [13]:
films2008 = df[df.release_year == 2008]
found = films2008[films2008.revenue == films2008.revenue.max()].iloc[0]

answers['9'] = format_movie(found)
print(answers['9'])

Вы думаете и в жизни у вас будут варианты ответов?. The Dark Knight (tt0468569)


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [14]:
films1214 = df[(2012 <= df.release_year) & (df.release_year <= 2014)]
found = films1214[films1214.profit == films1214.profit.min()].iloc[0]

answers['10'] = format_movie(found)
print(answers['10'])

Вы думаете и в жизни у вас будут варианты ответов?. The Lone Ranger (tt1210819)


# 11. Какого жанра фильмов больше всего?

In [15]:
all_genres = flat_map(df, lambda row: row.genres)
all_genres_counts = pd.Series(all_genres).value_counts()

answers['11'] = all_genres_counts.index[0]
print(answers['11'], all_genres_counts[0])


Drama 782


ВАРИАНТ 2

In [16]:
all_genres = []
for genres in df['genres']:
    for genre in genres:
        all_genres.append(genre)
    
all_genres_counts = pd.Series(all_genres).value_counts()

answers['11'] = all_genres_counts.index[0]
print(answers['11'], all_genres_counts[0])

Drama 782


# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [17]:
profit_movies = df[df.profit > 0]
profit_movies_genres = flat_map(profit_movies, lambda row: row.genres)
profit_movies_genres_counts = pd.Series(profit_movies_genres).value_counts()

answers['12'] = profit_movies_genres_counts.index[0]
print(answers['12'], profit_movies_genres_counts[0])

Drama 560


# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [18]:
revenues_by_director = Counter()

for row in df.itertuples():
    for director in row.director:
        revenues_by_director[director] += row.revenue

answers['13'] = revenues_by_director.most_common(1)[0][0]
print(answers['13'], revenues_by_director.most_common(1)[0][1])


Peter Jackson 6490593685


ВАРИАНТ 2

In [19]:
revenues_by_director = count_by_collection_item(df, lambda row: row.director, lambda row: row.revenue)

answers['13'] = revenues_by_director.most_common(1)[0][0]
print(answers['13'], revenues_by_director.most_common(1)[0][1])


Peter Jackson 6490593685


# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [20]:
action_movies = df[original_data.genres.str.contains('Action', na=False)]
action_directors = flat_map(action_movies, lambda row: row.director)
action_directors_counts = pd.Series(action_directors).value_counts()

answers['14'] = action_directors_counts.index[0]
print(answers['14'], action_directors_counts[0])


Robert Rodriguez 9


# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [21]:
revenue_by_actor = Counter()

for row in df[df.release_year == 2012].itertuples():
    for actor in row.cast:
        revenue_by_actor[actor] += row.revenue

top_revenue_actor = revenue_by_actor.most_common(1)[0]
answers['15'] = top_revenue_actor[0]
print(answers['15'], top_revenue_actor[1])

Chris Hemsworth 2027450773


ВАРИАНТ 2

In [22]:
revenue_by_actor = count_by_collection_item(df[df.release_year == 2012], lambda row: row.cast, lambda row: row.revenue)

top_revenue_actor = revenue_by_actor.most_common(1)[0]
answers['15'] = top_revenue_actor[0]
print(answers['15'], top_revenue_actor[1])

Chris Hemsworth 2027450773


# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [23]:
high_budget_movies = df[df.budget > df.budget.mean()]

high_budget_movies_by_actor = Counter()

for row in high_budget_movies.itertuples():
    for actor in row.cast:
        high_budget_movies_by_actor[actor] += 1

top_actor = high_budget_movies_by_actor.most_common(1)[0]
answers['16'] = top_actor[0]
print(answers['16'], top_actor[1])

Matt Damon 18


ВАРИАНТ 2

In [24]:
high_budget_movies = df[df.budget > df.budget.mean()]

high_budget_movies_by_actor = count_by_collection_item(high_budget_movies, lambda row: row.cast)

top_actor = high_budget_movies_by_actor.most_common(1)[0]
answers['16'] = top_actor[0]
print(answers['16'], top_actor[1])

Matt Damon 18


# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [25]:
the_movies = df[original_data.cast.str.contains('Nicolas Cage')]
the_genres = flat_map(the_movies, lambda row: row.genres)
the_genres_counts = pd.Series(the_genres).value_counts()

answers['17'] = the_genres_counts.index[0]
print(answers['17'], the_genres_counts[1])

Action 15


# 18. Самый убыточный фильм от Paramount Pictures

In [26]:
paramount_movies = df[original_data.production_companies.str.contains('Paramount Pictures')]
worst_movie = paramount_movies.sort_values(by='profit').iloc[0]

answers['18'] = format_movie(worst_movie)
print(answers['18'])

Вы думаете и в жизни у вас будут варианты ответов?. K-19: The Widowmaker (tt0267626)


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [27]:
annual_profits = df.groupby(['release_year'])['revenue'].sum().sort_values(ascending=False)

answers['19'] = annual_profits.index[0]
print(answers['19'], annual_profits.iloc[0])

2015 25449202382


# 20. Какой самый прибыльный год для студии Warner Bros?

In [28]:
warner_movies = df[original_data.production_companies.str.contains('Warner Bros')]
annual_profits = warner_movies.groupby(['release_year'])['profit'].sum().sort_values(ascending=False)

answers['20'] = annual_profits.index[0]
print(answers['20'], annual_profits.iloc[0])

2014 2295464519


# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [29]:
movies_by_month = df.groupby(['release_month'])['imdb_id'].count().sort_values(ascending=False)

answers['21'] = movies_by_month.index[0]
print(answers['21'], movies_by_month.iloc[0])

9 227


# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [30]:
summer_movies = df[df.release_month.isin([6, 7, 8])]

answers['22'] = len(summer_movies)
print(answers['22'])

450


# 23. Для какого режиссера зима – самое продуктивное время года? 

In [31]:
winter_movies = df[df.release_month.isin([12, 1, 2])]

winter_movies_directors = flat_map(winter_movies, lambda row: row.director)
winter_movies_directors_counts = pd.Series(winter_movies_directors).value_counts()
winter_movies_directors_counts

answers['23'] = winter_movies_directors_counts.index[0]
print(answers['23'], winter_movies_directors_counts.iloc[0])

Peter Jackson 7


# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [32]:
titles_by_studio = {}
title_size_by_studio = {}

for row in df.itertuples():
    for studio in row.production_companies:
        if studio not in titles_by_studio:
            titles_by_studio[studio] = []
        titles_by_studio[studio].append(row.original_title)
        
for studio in titles_by_studio:
    title_size_by_studio[studio] = pd.Series(list(map(lambda title: len(title), titles_by_studio[studio]))).mean()

title_size_by_studio = sort_dictionary_by_value(title_size_by_studio)

key = next(iter(title_size_by_studio))
answers['24'] = key
print(answers['24'], title_size_by_studio[key])

Four By Two Productions 83.0


# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [33]:
overviews_by_studio = {}
overviews_size_by_studio = {}

for row in df.itertuples():
    for studio in row.production_companies:
        if studio not in overviews_by_studio:
            overviews_by_studio[studio] = []
        overviews_by_studio[studio].append(row.overview)
        
for studio in overviews_by_studio:
    overviews_size_by_studio[studio] = pd.Series(list(map(lambda title: len(title), overviews_by_studio[studio]))).mean()

overviews_size_by_studio = sort_dictionary_by_value(overviews_size_by_studio)

key = next(iter(overviews_size_by_studio))
answers['25'] = key
print(answers['25'], overviews_size_by_studio[key])

Midnight Picture Show 1000.0


# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [34]:
# TODO: what is the better way to cut out 99% ?
# (moovies with rating 7.8 get inside both 1% and 99% ranges)
movies_count_in_percent = int(len(df) / 100)

top_movies = df.sort_values(by='vote_average', ascending=False).head(movies_count_in_percent)
display(list(top_movies['original_title']))

answers['26'] = 'Inside Out, The Dark Knight, 12 Years a Slave'
print(answers['26'])

['The Dark Knight',
 'Interstellar',
 'The Imitation Game',
 'Inside Out',
 'Room',
 'The Wolf of Wall Street',
 'Gone Girl',
 '12 Years a Slave',
 'Guardians of the Galaxy',
 'The Lord of the Rings: The Return of the King',
 'Memento',
 'Inception',
 'The Pianist',
 'The Grand Budapest Hotel',
 'Her',
 'Spotlight',
 'Big Hero 6',
 'The Fault in Our Stars']

Inside Out, The Dark Knight, 12 Years a Slave


# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [35]:
actor_pairs = Counter()

for row in df.itertuples():
    for actor1 in row.cast:
        for actor2 in row.cast:
            if (actor1 != actor2) & (actor1 < actor2):
                actor_pairs["{} & {}".format(actor1, actor2)] += 1

top_pairs = actor_pairs.most_common()

answers['27'] = top_pairs[0][0]
print(answers['27'], top_pairs[0][1])

Daniel Radcliffe & Rupert Grint 8


# Submission

In [36]:
answers

{'1': 'Вы думаете и в жизни у вас будут варианты ответов?. Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Вы думаете и в жизни у вас будут варианты ответов?. Gods and Generals (tt0279111)',
 '3': 'Вы думаете и в жизни у вас будут варианты ответов?. Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': 'Вы думаете и в жизни у вас будут варианты ответов?. Avatar (tt0499549)',
 '7': 'Вы думаете и в жизни у вас будут варианты ответов?. The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'Вы думаете и в жизни у вас будут варианты ответов?. The Dark Knight (tt0468569)',
 '10': 'Вы думаете и в жизни у вас будут варианты ответов?. The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'Вы думаете и в жизни у вас будут варианты ответов?. K-19: The Widowmaker (tt0267626)',
 '19': 2015,
 '20': 2014,
 '21': 9,
 '22': 450,
 '23': 'Peter Jackson',
 '24': 

In [37]:
len(answers)

27